Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import pyprind

## STEP 1 : Grab Data

In [2]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [3]:
from nltk.tokenize import sent_tokenize

ident = []
text = []
division = []
#DATA_PATH = [SOCIALE_PATH] # For faster test
DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)

            
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = "".join(child.itertext())
                text.append(sent_tokenize(contenu))
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)


## STEP 2 : Create DataFrame to manipulate easily data

In [11]:
if(df is None) :
    df = pd.read_pickle('3FD8KA7.pkl')
    
else :
    d = {'id': ident, 'text': text, 'division': division}
    df = pd.DataFrame(data = d)

    # remove all NaN values from df
    df.dropna()

    # save for the next time
    df.to_pickle('3FD8KA7.pkl')

In [12]:
fast_execution = True #to run code faster
mini_df = df.sample(500)

if(fast_execution) : 
    X = mini_df['text']
    y = mini_df['division']
    print('You chose to run using fast_execution')
    print('data size: ', X.shape[0])
else :
    X = df['text']
    y = df['division']
    print('You chose to run with the whole dataset')
    print('data size: ', X.shape[0])

You chose to run using fast_execution
data size:  500


## STEP 4 : Clean Data

In [13]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=Code )civil|de l\'action sociale et des familles|de l\'artisanat|des assurances|de l\'aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', ''.join(text))


X = X.apply(clean_references_to_law_codes)

In [14]:
def clean_references_to_law_articles(text) :
    return re.sub('(?:[A-Z][\.-]? ?)?(?:\d+-?\d+)', 'N', ''.join(text))

X = X.apply(clean_references_to_law_articles)

In [15]:
# Preprocessing step pull off all the htlm tag, "\n", and any special characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', ''.join(text))
    text = re.sub('[.,\/#!$%\^&\*;:{}=\-_`~()«»°–]','', ''.join(text))
    text = text.replace('\n','').replace('\'',' ')
    return text.replace('\s{2,}',' ')

X = X.apply(preprocessor)

In [16]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

X = X.apply(to_lower_case)

In [17]:
# We decided to replace any corresponding labels to a unique special label : *_* 
def clean_words_corresponding_to_labels(text) :
    return re.sub('criminel(le)?|commercial(e)?|social(e)?|civil(e)?', '*_*', ''.join(text))

X = X.apply(clean_words_corresponding_to_labels)

In [18]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def remove_stop_words(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(''.join(text), language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

X = X.apply(remove_stop_words)

In [12]:
X

37735     premier moyen attendu resulte arret infirmatif...
116452    moyen unique attendu x administrateur société ...
5734      cour cassation troisième chambre *_* a rendu a...
63253     moyen unique vu les articlen code general impo...
38252     moyen unique pris deux branches attendu selon ...
80388     cour cassation chambre *_* audience publique t...
83305     rejet pourvoi x contre arret cour appel paris ...
104929    moyen unique vu les articles suivants code tra...
100016    cour cassation chambre *_* a rendu arrêt suiva...
42665     les trois moyens reunis attendu aa fait appel ...
107368    les deux moyens reunis pris violation articlen...
64379     premier moyen vu articln code procedure *_* at...
48129     deuxieme moyen vu article 6 ordonnance dn octo...
28082     nom peuple francais cour cassation deuxieme ch...
128633    premier moyen attendu fait grief a arret attaq...
127707    moyen unique attendu fait grief a arret attaqu...
61072     cour cassation chambre *_* a r

## Most influential words

In [19]:
X_civile = X.where(y=='CIVILE')
X_commerciale = X.where(y=='COMMERCIALE')
X_criminelle = X.where(y=='CRIMINELLE')
X_sociale = X.where(y=='SOCIALE')

#to remove NaN values
X_civile = X_civile[~X_civile.isnull()]
X_commerciale = X_commerciale[~X_commerciale.isnull()]
X_criminelle = X_criminelle[~X_criminelle.isnull()]
X_sociale = X_sociale[~X_sociale.isnull()]

In [20]:
#so they all have differents shapes
print(X_civile.shape[0])
print(X_commerciale.shape[0])
print(X_criminelle.shape[0])
print(X_sociale.shape[0])

224
74
65
137


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#vec_pipe = Pipeline([
#    ('vect', CountVectorizer(preprocessor = preprocessor)),
#    ('tfidf', TfidfTransformer())
#])

#vec_pipe.fit(X)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1),
        preprocessor=<function preprocessor at 0x...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])

In [17]:
#vec = vec_pipe.named_steps['vect']
#feature_array = np.array(vec.get_feature_names())

In [19]:
def get_influential_words(X, vec_pipe, feature_array, top=10) : 
    Xtr = vec_pipe.transform(X)
    
    #feature_array = np.array(vec.get_feature_names())
    tfidf_sorting = np.argsort(Xtr.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:top]
    
    return top_n

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def get_influential_words(X, top=10) :
    count_vect = CountVectorizer()
    count_vect = count_vect.fit(X)
    freq_term_matrix = count_vect.transform(X)

    tfidf = TfidfTransformer(norm="l2")
    tfidf.fit(freq_term_matrix)
    
    doc_freq_term = count_vect.transform(X)
    doc_tfidf_matrix = tfidf.transform(doc_freq_term)
    
    tfidf_sorting = np.argsort(doc_tfidf_matrix.toarray()).flatten()[::-1]
    feature_array = np.array(count_vect.get_feature_names())
    top_n = feature_array[tfidf_sorting][:top]
    
    return top_n

In [36]:
get_influential_words(X_sociale)

array(['conclue', 'charte', 'créent', 'adressées', 'espèce', 'esthétique',
       'demeurer', 'aoutque', 'aout', 'barbara'], dtype='<U39')

In [30]:
top_civile = get_influential_words(X_civile)
top_commerciale = get_influential_words(X_commerciale)
top_criminelle = get_influential_words(X_criminelle)
top_sociale = get_influential_words(X_sociale)

In [24]:
print(top_commerciale)

['grottes' 'latreille' 'soussol' 'nu' 'denie' 'exploitation' 'septembrn'
 'decret' 'locataires' 'parce']


In [31]:
per_division = {'civile': top_civile, 'commerciale': top_commerciale, 'criminelle': top_criminelle, 'sociale': top_sociale}
words_df = pd.DataFrame(data = per_division)

words_df

,civile,commerciale,criminelle,sociale
0,mandat,grands,majorite,contrôle
1,les,moulins,organisations,redressement
2,epoux,epoux,prefet,avril
3,special,dole,volonte,agent
4,echangees,les,accord,observations
5,consorts,piece,etait,rapport
6,lettres,versee,arret,accompagné
7,memes,ni,arrete,reçue
8,clients,preuve,travail,société
9,meme,cinq,meubles,réponse


## Build document summarizer

Once you'll build the tf-idf matrix for the text corpus, you will use the tf-idf of each word to compute a value for each sentence. The n top sentences for a document will be used to represent the document.

## STEP 3 : Split Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

## STEP 5 : Train the model

In [21]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [22]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

ValueError: could not convert string to float: "\nSur le moyen unique du pourvoi incident, qui est préalable :\n   Attendu, selon l'arrêt attaqué qu'un tribunal de grande instance a prononcé la séparation de corps des époux X... aux torts de l'épouse ; que, par assignation du 21 juillet 1989, le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce ; que, par assignation en date du 2 août 1989, Mme X..., autorisée par une ordonnance de non-conciliation du 19 janvier 1989, a assigné son mari en divorce pour faute ; qu'après jonction de ces deux procédures, le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme, et, accueillant la demande en divorce de celle-ci, prononcé également le divorce à son profit ; qu'ensuite le Tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés ;\n   Attendu qu'il est fait grief à l'arrêt infirmatif attaqué d'avoir déclaré irrecevable la demande en 

In [153]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

gdb_clf = GradientBoostingClassifier(random_state=42, n_estimators=100)

voting_clf = VotingClassifier(
    estimators=[('rfc', rnd_clf), ('abc', ada_clf), ('gbr', gdb_clf)],
    voting='hard')

In [154]:
for clf in (rnd_clf, ada_clf, gdb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'sur le moyen unique du pourvoi incident qui est préalable    attendu selon l arrêt attaqué qu un tribunal de grande instance a prononcé la séparation de corps des époux x aux torts de l épouse  que par assignation du 21 juillet 1989 le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce  que par assignation en date du 2 août 1989 mme x autorisée par une ordonnance de nonconciliation du 19 janvier 1989 a assigné son mari en divorce pour faute  qu après jonction de ces deux procédures le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme et accueillant la demande en divorce de celleci prononcé également le divorce à son profit  qu ensuite le tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés    attendu qu il est fait grief à l arrêt infirmatif attaqué d avoir déclaré irrecevable la demande en divorce de la femme alors que d 